In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K

import tensorflow as tf
from sklearn.preprocessing import normalize
from tensorflow.keras.layers import Input, Dense, Activation,Lambda,Flatten,Dropout, Concatenate
from tensorflow.keras import Model
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH = "/content/drive/My Drive/PROJET F4 ZZ3 HAMZA/bounding box prediction/"

In [4]:
X = np.float64(np.load(PATH+"bbox_camera.npy"))
Y = np.float64(np.load(PATH+"bbox_radar.npy"))

In [5]:
X.shape

(2065, 4)

In [6]:
X[len(X)-1]

array([1017.,  576., 1279., 1059.])

In [7]:
X[0]

array([579., 503., 531., 410.])

In [8]:
X_fit_scaler = MinMaxScaler().fit(X)
X_scaler = X_fit_scaler.transform(X)

Y_fit_scaler = MinMaxScaler().fit(Y)
Y_scaler = Y_fit_scaler.transform(Y)

In [9]:
X_scaler[0]

array([0.22513089, 0.34671533, 0.39334955, 0.33064516])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaler, Y_scaler, test_size=0.2, random_state=42)

In [11]:
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(4))

lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.000001)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])
history = model.fit(X_train, y_train, epochs=60, batch_size=10, validation_data=(X_test,y_test), callbacks=[lr_reduction])

Epoch 1/60
166/166 [==============================] - 3s 12ms/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.0077 - val_mean_squared_error: 0.0077 - lr: 0.0010
Epoch 2/60
166/166 [==============================] - 2s 10ms/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0065 - val_mean_squared_error: 0.0065 - lr: 0.0010
Epoch 3/60
166/166 [==============================] - 2s 10ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0061 - val_mean_squared_error: 0.0061 - lr: 0.0010
Epoch 4/60
166/166 [==============================] - 2s 11ms/step - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0080 - val_mean_squared_error: 0.0080 - lr: 0.0010
Epoch 5/60
166/166 [==============================] - 2s 10ms/step - loss: 0.0057 - mean_squared_error: 0.0057 - val_loss: 0.0047 - val_mean_squared_error: 0.0047 - lr: 0.0010
Epoch 6/60
166/166 [==============================] - 2s 11ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_los

In [13]:
model.save('/content/drive/My Drive/PROJET F4 ZZ3 HAMZA/FCNN_Bounding_box.h5')

In [12]:
Y_pred = model.predict(X_test)

In [14]:
Y_pred_original = Y_fit_scaler.inverse_transform(Y_pred)
Y_test_original = Y_fit_scaler.inverse_transform(y_test)

In [15]:
mean_squared_error(Y_test_original[:,0], Y_pred_original[:,0], squared=False)

21.460667200519037

In [16]:
mean_squared_error(Y_test_original[:,1], Y_pred_original[:,1], squared=False)

22.103907983683943

In [17]:
mean_squared_error(Y_test_original[:,2], Y_pred_original[:,2], squared=False)

4.928197941797449

In [18]:
mean_squared_error(Y_test_original[:,3], Y_pred_original[:,3], squared=False)

5.00509444174477